# **Feature engineering**

# Variables dérivées

## Variables dérivées des `SK`

### `is_in_*`

Toute clé `SK_ID_CURR` présente dans l'une des tables correspond à un enregistrement unique dans la table `applications`. La réciproque est fausse.

Ainsi, nous formons 5 variables binaires additionnelles qui étendent la table `application` en informant sur la présence du client demandeur dans l'une ou l'autre des 5 sous-tables.

**TODO** ajouter également `bureau_balance` : un client suivi par `bureau` n'a pas nécessairement de détails dans `bureau_balance`.

Même approche avec la table `bureau` et sa clé primaire `SK_ID_BUREAU` et avec la table `previous_application` et sa clé primaire `SK_ID_PREV`.

#### `application`.`SK_ID_CURR`.`is_in_*`

In [4]:
from home_credit.feat_eng import is_in_ext
subs_table_names = [
    "bureau", "previous_application", "pos_cash_balance",
    "credit_card_balance", "installments_payments"
]
app_is_in = is_in_ext("SK_ID_CURR", "application", subs_table_names)
display(app_is_in)

,in_B,in_PA,in_PCB,in_CCB,in_IP
0,True,True,True,False,True
1,True,True,True,False,True
2,True,True,True,False,True
3,False,True,True,True,True
4,True,True,True,False,True
...,...,...,...,...,...
356250,True,True,True,False,True
356251,False,True,True,False,True
356252,True,True,True,False,True
356253,True,True,True,False,True


Réalisons un profilage proche similaire à celui des NA :

In [5]:
from pepper.feat_eng import get_profile
from pepper.univar import agg_value_counts
app_is_in_profile = get_profile(app_is_in)
display(agg_value_counts(app_is_in_profile))

,count,proportion
◻◻◻◼◻,201137,0.564587
◻◻◻◻◻,87257,0.244928
◼◻◻◼◻,34137,0.095822
◼◻◻◻◻,13034,0.036586
◻◼◼◼◼,12931,0.036297
◼◼◼◼◼,2360,0.006624
◻◻◼◻◻,1782,0.005002
◻◼◻◼◻,854,0.002397
◻◻◻◻◼:◼◼◼◻◼,2763,0.007756


#### `bureau`.`SK_ID_BUREAU`.`is_in_*`

In [10]:
from home_credit.feat_eng import is_in_ext
bur_is_in = is_in_ext("SK_ID_BUREAU", "bureau", ["bureau_balance"])
display(bur_is_in)

,in_BB
0,False
1,False
2,False
3,False
4,False
...,...
1716423,False
1716424,False
1716425,False
1716426,False


In [9]:
from pepper.feat_eng import get_profile
from pepper.univar import agg_value_counts
bur_is_in_profile = get_profile(bur_is_in)
display(agg_value_counts(bur_is_in_profile))

,count,proportion
◼,942074,0.548857
◻,774354,0.451143


#### `previous_application`.`SK_ID_PREV`.`is_in_*`

In [12]:
from home_credit.feat_eng import is_in_ext
subs_table_names = [
    "pos_cash_balance", "credit_card_balance", "installments_payments"
]
prev_app_is_in = is_in_ext("SK_ID_PREV", "previous_application", subs_table_names)
display(prev_app_is_in)

,in_PCB,in_CCB,in_IP
0,True,False,True
1,True,False,True
2,True,False,True
3,True,False,True
4,False,False,False
...,...,...,...
1670209,True,False,True
1670210,True,False,True
1670211,True,False,True
1670212,True,False,True


Observons que :
* les cas `◻◻◻` et `◻◻◼` n'existent pas,
* les cas `◻◼◼` et `◼◼◻` sont marginaux,
* les cas `◼◻◻` et `◼◻◼` représentent chacun moins de $5\,\%$,
* les cas `◻◼◻` et `◼◼◼` se partagent le gros du batillon avec respectivement $40\,\%$ et $54\,\%$.

En d'autres termes, l'effectif client est principalement composé des utilisateurs de la carte de crédit ($94\,\%$).

En outre, près de $60\,\%$ de cet effectif est également consommateur des deux autres produits, le POS Cash et le Prêt. 

In [14]:
from pepper.feat_eng import get_profile
from pepper.univar import agg_value_counts
prev_app_is_in_profile = get_profile(prev_app_is_in)
display(agg_value_counts(prev_app_is_in_profile, agg=None))

,count,proportion
◻◼◻,896302,0.536639
◼◼◼,678044,0.405962
◼◻◻,62271,0.037283
◼◻◼,30664,0.018359
◻◼◼,2601,0.001557
◼◼◻,332,0.000199


### Nombre d'éléments groupés

Pour chaque niveau de synthèse effectué, par exemple la réduction des détails mensuels dans les tables de niveau 3 (`bureau_balance`, `pos_cash_balance`, `credit_card_balance`, `installments_payments`), la réduction du nombre d'anciens crédits suivis au niveau 2 (`bureau`, `previous_application`), il serait dommage de perdre l'information du nombre de lignes groupées par l'étape de synthèse effectuée. Cette information peut se réveler pertinente vis à vis de l'objectif de prédiction, ce qui ne peut être anticipé à ce stade. Elle peut aussi être indispensable simplement dans le cadre du travail de feature engineering. Par exemple, si nous dispons ici d'une somme, là d'un décompte particulier, ils pourront être relativisés par rapport à ce nombre conservé.